# Simulate the observations
Run these in a handful of filters and with/without coronagraphy.
Here we assume that the scene has been created elsewhere.

In [1]:
import json
import matplotlib.pyplot as plt
import copy
import numpy as np
import multiprocessing as mp
import pickle

import pandisk as pd
import pandeia.engine
import jwst_pancake as pc

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# base observation config
cor_base = pandeia.engine.calc_utils.build_default_calc('jwst', 'miri', 'coronagraphy')
cor_base['scene'].clear()

img_base = pandeia.engine.calc_utils.build_default_calc('jwst', 'miri', 'imaging')
img_base['scene'].clear()

In [3]:
# lists of setups, change number of groups to limit imaging saturation,
# but avoid going below 5
# https://jwst-docs.stsci.edu/display/JPP/MIRI+Generic+Recommended+Strategies
img_sub = 'sub256' #30" sqaure
subarrays  = [ img_sub, 'mask1065',  img_sub, 'mask1550',  img_sub, 'masklyot',  img_sub,  img_sub]
apertures  = ['imager', 'fqpm1065', 'imager', 'fqpm1550', 'imager', 'lyot2300', 'imager', 'imager']
filters    = ['f1000w', 'f1065c',   'f1500w', 'f1550c',   'f1800w', 'f2300c',   'f2100w', 'f2550w']
targ_ngrps = [ 5,        100,        7,        100,        7,        100,        7,        10]
ref_ngrps  = [ 5,        100,        5,        100,        5,        100,        5,        10]

# copy the above and delete as appropriate
subarrays  = ['mask1065',  img_sub, 'mask1550', 'masklyot',  img_sub]
apertures  = ['fqpm1065', 'imager', 'fqpm1550', 'lyot2300', 'imager']
filters    = ['f1065c',   'f1500w', 'f1550c',   'f2300c',   'f2550w']
targ_ngrps = [ 100,        7,        100,        100,        10]
ref_ngrps  = [ 100,        5,        100,        100,        10]

# nint will be this/ngroups, 10,000 is approx 40-50 minutes
# for MIRI coron and sub256 subarrays
target_int = 10000
ref_int    = 10000

In [4]:
# get the scene - this is the science object
scene_dir = './' #'../../../cycle1/hd38858/'
with open(scene_dir+'targ.json','r') as f:
    targ_scene = json.load(f)
    
# add in a global offset to capture the effect of target acquisition error.
errx, erry = pc.scene.get_ta_error()
pc.scene.offset_scene(targ_scene,errx,erry)

# same for reference star
ref_scene = copy.deepcopy(targ_scene)
with open(scene_dir+'ref.json','r') as f:
    ref_scene = json.load(f)
    
# And add target acquisition error
errx_ref, erry_ref = pc.scene.get_ta_error()
pc.scene.offset_scene(ref_scene,errx_ref,erry_ref)

In [5]:
# make a list, one element per observation
target = []
targ_star = []
ref_star = []
for suba, aper, filt, tngrp, rngrp in zip(subarrays, apertures, filters, targ_ngrps, ref_ngrps):

    if aper == 'imager':
        targ = copy.deepcopy(img_base)
        targ['configuration']['dynamic_image'] = True
        targ['configuration']['scene_size'] = 15.0
    else:
        targ = copy.deepcopy(cor_base)
        
    # target
    targ['configuration']['detector']['subarray'] = suba
    targ['configuration']['instrument']['aperture'] = aper
    targ['configuration']['instrument']['filter'] = filt
    targ['configuration']['detector']['ngroup'] = tngrp
    targ['configuration']['detector']['nint'] = target_int / tngrp
    targ['configuration']['detector']['readmode'] = 'fast'
    targ['scene'] = copy.deepcopy(targ_scene)

    # target star by itself
    star = copy.deepcopy(targ)
    star['scene'] = star['scene'][:1]
    
    # reference star
    ref = copy.deepcopy(targ)
    ref['configuration']['detector']['ngroup'] = rngrp
    ref['configuration']['detector']['nint'] = ref_int / rngrp
    ref['scene'] = copy.deepcopy(ref_scene)
    
    target.append(targ)
    targ_star.append(star)
    ref_star.append(ref)

In [6]:
# additional calculation config, if we're using pandeia_coronagraphy
pc.engine.options.wave_sampling = 10        # set ~10 for speed, >50 for accuracy
pc.engine.options.on_the_fly_PSFs = False   # True to get on-the-fly PSFs, only works for coron

In [7]:
# pandeia calculation, pandeia-coronagraphy package includes realistic noise 
# by not fixing a random seed
def calculate_batch(calcfiles, nprocesses=None):
    if nprocesses is None:
        nprocesses = mp.cpu_count()
    pool = mp.Pool(processes = nprocesses)
#     results = pool.map(pandeia.engine.perform_calculation.perform_calculation, calcfiles)
    results = pool.map(pc.engine.perform_calculation, calcfiles)
    pool.close()
    pool.join()
    return results

In [ ]:
# do the calculation, gives quite a few warnings
results = calculate_batch(target+targ_star+ref_star)
target_results = results[:len(target)]
star_results = results[len(target):len(target)+len(targ_star)]
ref_results = results[len(target)+len(targ_star):]

/usr/local/miniconda3/envs/astroconda/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: overflow encountered in multiply
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/usr/local/miniconda3/envs/astroconda/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: overflow encountered in multiply
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/usr/local/miniconda3/envs/astroconda/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: overflow encountered in multiply
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/usr/local/miniconda3/envs/astroconda/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: overflow encountered in multiply
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/usr/local/miniconda3/envs/astroconda/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: overflow encoun

In [ ]:
# extract some relevant images
target_slope = []
target_snr = []
target_sat = []
star_slope = []
star_snr = []
star_sat = []
ref_slope = []
ref_snr = []
ref_sat = []
for t, s, r in zip(target_results, star_results, ref_results):
    target_slope.append(t['2d']['detector'])
    target_snr.append(t['2d']['snr'])
    target_sat.append(t['2d']['saturation'])
    star_slope.append(s['2d']['detector'])
    star_snr.append(s['2d']['snr'])
    star_sat.append(s['2d']['saturation'])
    ref_slope.append(r['2d']['detector'])
    ref_snr.append(r['2d']['snr'])
    ref_sat.append(r['2d']['saturation'])

# Explore the results
PSF is not properly generated for imaging.

In [ ]:
# target slope images
pd.show_images(target_slope, title=list(zip(filters,np.tile('e/s',len(filters)))))
pd.show_images(target_snr, title=list(zip(filters,np.tile('snr',len(filters)))), log=False)
pd.show_images(target_snr, title=list(zip(filters,np.tile('snr',len(filters)))), log=True)
pd.show_images(target_sat, title=list(zip(filters,np.tile('sat',len(filters)))))

In [ ]:
# star slope images
pd.show_images(star_slope, title=list(zip(filters,np.tile('e/s',len(filters)))))
pd.show_images(star_snr, title=list(zip(filters,np.tile('snr',len(filters)))))
pd.show_images(star_sat, title=list(zip(filters,np.tile('sat',len(filters)))))

In [ ]:
# reference slope images
pd.show_images(ref_slope, title=list(zip(filters,np.tile('e/s',len(filters)))))
pd.show_images(ref_snr, title=list(zip(filters,np.tile('snr',len(filters)))))
pd.show_images(ref_sat, title=list(zip(filters,np.tile('sat',len(filters)))))

In [ ]:
# subtraction in perfect case; reference is target star
pd.show_images(target_slope, sub=star_slope, title=list(zip(filters,np.tile('perfect sub',len(filters)))))
pd.show_images(target_slope, sub=star_slope, title=list(zip(filters,np.tile('perfect sub',len(filters)))), log=True)

In [ ]:
# reference subtraction without recentering
norm = [np.nanmax(target_slope[i])/np.nanmax(ref_slope[i]) for i in range(len(target_slope))]
sub = [r*n for r,n in zip(ref_slope, norm)]

pd.show_images(target_slope, sub=sub, title=list(zip(filters,np.tile('psf sub',len(filters)))))

In [ ]:
# registered subtraction
reg_ref = []
target_bg = []
for t,r in zip(target_slope, ref_slope):
    clean_targ = t.copy()
    clean_targ[np.isnan(t)] = np.nanmax(t)
    clean_targ -= np.percentile(clean_targ, 30)
    clean_ref = r.copy()
    clean_ref[np.isnan(r)] = np.nanmax(r)
    clean_ref -= np.percentile(clean_ref, 30)
    reg_ref.append(pc.analysis.register_to_target(clean_ref, clean_targ, rescale_reference=True))
    target_bg.append(clean_targ - np.percentile(clean_targ, 30))

pd.show_images(target_bg, sub=reg_ref, title=list(zip(filters,np.tile('psf sub',len(filters)))))
pd.show_images(target_bg, sub=reg_ref, title=list(zip(filters,np.tile('psf sub',len(filters)))), log=True)

In [ ]:
# save the results
with open('pandeia-results.pkl','wb') as f:
    pickle.dump(results, f)